**Here is my notebook to solve this task using simple dnn (dense neural network)!**

# Import libs

In [ ]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [ ]:
train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')

train.head(3)

During all experiments I found out the best way to preprocess the text. Somehow non alphabet characters doens't need to be deleted.

In [ ]:
def text_preprocessing(text):
    tokenized = word_tokenize(text)
    text = [i for i in tokenized if i not in stopwords.words('english')]
    text = ' '.join(text)
    
    return text

In [ ]:
x = train.excerpt.apply(text_preprocessing)
y = train.target

In [ ]:
tokenizer = Tokenizer()

# fit only train set
tokenizer.fit_on_texts(x)

# in case you have validation/test dataset don't forget to transform val/test data on already fitted train data
x = tokenizer.texts_to_sequences(x)

In [ ]:
# find out the longest sequence length for padding
len_seq_list = [len(s) for s in x]
max_seq_len = np.max(len_seq_list)

# pad sequence for first embedding layer
x = tf.keras.preprocessing.sequence.pad_sequences(
    x,
    padding='post',
    truncating='post',
    maxlen=max_seq_len
)

In [ ]:
# hyperparameters
voc_size = len(tokenizer.index_word) + 1
epochs = 100
batch_size = 1024
embedding_dim = 512
dropout_rate = .5

# Build model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(voc_size, embedding_dim, input_length=max_seq_len),
#     gap1d layer shows better accuracy than flatten layer
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.summary()

Decent callbacks to control train/val loss to not let the model to overfit.

In [ ]:
early_stopping = EarlyStopping(patience=8, restore_best_weights=True, verbose=1)
lr_reduce = ReduceLROnPlateau(patience=3, verbose=1)

callbacks = [early_stopping, lr_reduce]

# Training

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='mean_squared_error'
)

history = model.fit(
    x,
    y,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=.2,
    callbacks=callbacks
)

# Make a submission file

In [ ]:
sub = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
test_data = test.excerpt.apply(text_preprocessing)
test_data = tokenizer.texts_to_sequences(test_data)
test_data = tf.keras.preprocessing.sequence.pad_sequences(
    test_data,
    padding='post',
    truncating='post',
    maxlen=max_seq_len
)

test_pred = model.predict(test_data)

In [ ]:
sub['target'] = test_pred
sub.to_csv('submission.csv', index=False)

Thanks for watching!

If you have any *questions* or *suggestions* feel free to write 'em down below!